In [83]:
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np

import os
import sys
import re
import random
import copy
from shutil import copytree
from PIL import Image


from IPython.display import display, Image

from scipy import ndimage
from scipy.misc import imresize
import cv2

from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

%matplotlib inline

plt.xkcd()

"""
Set the parameter here, your root and the class you want to neglect.
Classlist:
Plants, Paved ground, Outdoor structures, Non-paved ground, Buildings, Wheeled Vehicles,
People, Water, Train tracks.
"""

ROOT = '/Users/jeffshih/Desktop/Programming/DroneSegmentation/datasets/Okutama'
global CLASSFLAG 
CLASSFLAG = 'People'
global SUBFLAG
SUBFLAG='Background'
os.chdir(ROOT)
print('Root datasets directory path: ', ROOT)



Root datasets directory path:  /Users/jeffshih/Desktop/Programming/DroneSegmentation/datasets/Okutama


In [84]:
"""
Read in the class mapping and the certain class index
"""

def read_class_mapping(class_file,CLASSFLAG,SUBFLAG):
    with open(class_file, 'r') as f:
        class_mapping = {}
        labels = []
        for idx, line in enumerate(f):
            labels.append(line.split(':')[1].strip(' \t\n\r'))
            class_mapping[labels[-1]] = idx
        return labels.index(CLASSFLAG),labels.index(SUBFLAG)

global classIndex 
global subIndex 

classIndex,subIndex = read_class_mapping(os.path.join(ROOT, 'Class_list.txt'),CLASSFLAG,SUBFLAG)



print(classIndex,subIndex)



3 0


In [107]:
def create_gt_list(gt_folders):
    imgs = []
    for root,subdir,files in os.walk(gt_folders):
            for filename in files:
                imgs.append(filename)
    return imgs

def calc_class_hist(img):
    return cv2.calcHist([img],[0],None,[10],[0,10])

def replace_class(gtFolders,gtList):
    histBefore = [0]
    histAfter = [0]
    for imgName in gtList:
        imgPath = os.path.join(gtFolders,imgName)
        img = cv2.imread(imgPath,cv2.IMREAD_GRAYSCALE)
        histBefore = np.add(histBefore,calc_class_hist(img))
        subImg = replace_class_img(img,classIndex,subIndex)
        write_image(img,imgName)
        histAfter = np.add(histAfter,calc_class_hist(subImg))
    return histBefore,histAfter
def replace_class_img(img,classIndex,subIndex):
    img[img==classIndex]=subIndex
    return img

def write_image(img,imgName):
    new_gt_dir = os.path.join(ROOT,'substitute_ground_truth')
    new_gt_file = os.path.join(new_gt_dir,imgName)
    #print(new_gt_file)
    if not os.path.exists(new_gt_dir):
        os.makedirs(new_gt_dir)
    #cv2.imwrite(new_gt_file,img)

    

gtFolders = os.path.join(ROOT,'ground_truth')
#print(gt_folders)
gtList = create_gt_list(gtFolders)
histBefore,histAfter = replace_class(gtFolders,gtList)

print(histBefore)
print(histAfter)
#plt.hist(histAfter)


new_gt_dir = os.path.join(ROOT,'substitute_ground_truth')





[[  7.01912340e+07]
 [  2.01153930e+07]
 [  1.49937368e+08]
 [  1.64684024e+08]
 [  6.84986150e+07]
 [  4.45956800e+06]
 [  2.61451328e+08]
 [  5.91889600e+06]
 [  9.47522800e+06]
 [  5.87460000e+04]]
[[  2.34875258e+08]
 [  2.01153930e+07]
 [  1.49937368e+08]
 [  0.00000000e+00]
 [  6.84986150e+07]
 [  4.45956800e+06]
 [  2.61451328e+08]
 [  5.91889600e+06]
 [  9.47522800e+06]
 [  5.87460000e+04]]
